*Pipeline Created by Justin Mumma - Pravetoni Lab, UW Medicine*

*Under Construction. Last updated: 02-27-2026*

---

# Setup Instructions
- Download legacy .pdb file for mAb + ligand complex (such as 7U64 for fentanyl).

**Important:**
- you must remove all water molecules, ions, and the assymetric unit from the .pdb structure *(using PyMol)*
---

#### How to clean and Structure the .pdb file:
- in PyMol command line, strip away unnecessary artifacts by typing:

  - `remove solvent` (deletes all water)
  - `remove inorg` (deletes all salts/ions)

- Isolate a single Fab-Fentanyl complex.
  - 7U64 usually has Chains A & B (one Fab) and Chains C & D (the second Fab), along with two fentanyl ligands.
  - Identify which fentanyl is bound to Chains A & B.
  - Delete Chains C, D, and the second fentanyl.

- Renaming Chains (**important**):
  - Colab script's contig (H1-97/.../L1-214) specifically looks for chain letters H and L. If 7U64 uses A and B, the script will crash.
  - Determine which chain is Heavy and which is Light (usually Heavy is the longer one, around 220 amino acids).
  - In PyMOL, use the alter command to rename them:

    `alter chain A, chain="H"`

    `alter chain B, chain="L"`
  - Rename the fentanyl chain to something unique, like `chain="Z"`
- Save this cleaned complex as `fent_mAb_7U64_clean.pdb`

  *(File > Export Molecule > Save as PDB)*,
  
  and upload to Colab




# RFDiffusion Partial Diffusion / Motif Scaffolding


*   Input your existing antibody structure but tell RFDiffusion to "mask" or delete specific CDR loops. u]Usually the heavy chain CDR3, which is most responsible for binding.
*    Provide the fentanyl molecule coordinates, keep the rest of the antibody rigid, and ask RFDiffusion to generate new backbone shapes only for that missing loop to better cradle the fentanyl.


*   **Next Step:** You pass those new backbone structures to LigandMPNN to generate the amino acid sequences for those new loops



In [ ]:
#@title setup **RFdiffusion** (~3min)
%%time
import os, time, signal
import sys, random, string, re
if not os.path.isdir("params"):
  os.system("apt-get install aria2")
  os.system("mkdir params")
  # send param download into background
  os.system("(\
  aria2c -q -x 16 https://files.ipd.uw.edu/krypton/schedules.zip; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt; \
  aria2c -q -x 16 http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt; \
  aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar; \
  tar -xf alphafold_params_2022-12-06.tar -C params; \
  touch params/done.txt) &")

if not os.path.isdir("RFdiffusion"):
  print("installing RFdiffusion...")
  os.system("git clone https://github.com/sokrypton/RFdiffusion.git")
  os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
  os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
  # 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
  # 25Aug2025: updating dgi install to work with latest pytorch
  os.system("pip install --no-dependencies dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html")
  os.system("pip install --no-dependencies e3nn==0.5.5 opt_einsum_fx")
  os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
  os.system("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
  os.system("chmod +x ananas")

if not os.path.isdir("colabdesign"):
  print("installing ColabDesign...")
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")

if not os.path.isdir("RFdiffusion/models"):
  print("downloading RFdiffusion params...")
  os.system("mkdir RFdiffusion/models")
  models = ["Base_ckpt.pt","Complex_base_ckpt.pt","Complex_beta_ckpt.pt"]
  for m in models:
    while os.path.isfile(f"{m}.aria2"):
      time.sleep(5)
  os.system(f"mv {' '.join(models)} RFdiffusion/models")
  os.system("unzip schedules.zip; rm schedules.zip")

if 'RFdiffusion' not in sys.path:
  os.environ["DGLBACKEND"] = "pytorch"
  sys.path.append('RFdiffusion')

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    if not os.path.isfile(f"{pdb_code}.pdb1"):
      os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz")
      os.system(f"gunzip {pdb_code}.pdb1.gz")
    return f"{pdb_code}.pdb1"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

def run_ananas(pdb_str, path, sym=None):
  pdb_filename = f"outputs/{path}/ananas_input.pdb"
  out_filename = f"outputs/{path}/ananas.json"
  with open(pdb_filename,"w") as handle:
    handle.write(pdb_str)

  cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
  if sym is None: os.system(cmd)
  else: os.system(f"{cmd} {sym}")

  # parse results
  try:
    out = json.loads(open(out_filename,"r").read())
    results,AU = out[0], out[-1]["AU"]
    group = AU["group"]
    chains = AU["chain names"]
    rmsd = results["Average_RMSD"]
    print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

    C = np.array(results['transforms'][0]['CENTER'])
    A = [np.array(t["AXIS"]) for t in results['transforms']]

    # apply symmetry and filter to the asymmetric unit
    new_lines = []
    for line in pdb_str.split("\n"):
      if line.startswith("ATOM"):
        chain = line[21:22]
        if chain in chains:
          x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
          if group[0] == "c":
            x = sym_it(x,C,A[0])
          if group[0] == "d":
            x = sym_it(x,C,A[1],A[0])
          coord_str = "".join(["{:8.3f}".format(a) for a in x])
          new_lines.append(line[:30]+coord_str+line[54:])
      else:
        new_lines.append(line)
    return results, "\n".join(new_lines)

  except:
    return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

  def run_command_and_get_pid(command):
    pid_file = '/dev/shm/pid'
    os.system(f'nohup {command} & echo $! > {pid_file}')
    with open(pid_file, 'r') as f:
      pid = int(f.read().strip())
    os.remove(pid_file)
    return pid
  def is_process_running(pid):
    try:
      os.kill(pid, 0)
    except OSError:
      return False
    else:
      return True

  run_output = widgets.Output()
  progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
  display(widgets.VBox([progress, run_output]))

  # clear previous run
  for n in range(steps):
    if os.path.isfile(f"/dev/shm/{n}.pdb"):
      os.remove(f"/dev/shm/{n}.pdb")

  pid = run_command_and_get_pid(command)
  try:
    fail = False
    for _ in range(num_designs):

      # for each step check if output generated
      for n in range(steps):
        wait = True
        while wait and not fail:
          time.sleep(0.1)
          if os.path.isfile(f"/dev/shm/{n}.pdb"):
            pdb_str = open(f"/dev/shm/{n}.pdb").read()
            if pdb_str[-3:] == "TER":
              wait = False
            elif not is_process_running(pid):
              fail = True
          elif not is_process_running(pid):
            fail = True

        if fail:
          progress.bar_style = 'danger'
          progress.description = "failed"
          break

        else:
          progress.value = (n+1) / steps
          if visual != "none":
            with run_output:
              run_output.clear_output(wait=True)
              if visual == "image":
                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                fig = plt.figure()
                fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1)
                plt.show()
              if visual == "interactive":
                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                view.addModel(pdb_str,'pdb')
                view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                view.zoomTo()
                view.show()
        if os.path.exists(f"/dev/shm/{n}.pdb"):
          os.remove(f"/dev/shm/{n}.pdb")
      if fail:
        progress.bar_style = 'danger'
        progress.description = "failed"
        break

    while is_process_running(pid):
      time.sleep(0.1)

  except KeyboardInterrupt:
    os.kill(pid, signal.SIGTERM)
    progress.bar_style = 'danger'
    progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, partial_T="auto",
                  num_designs=1, use_beta_model=False, visual="none"):

  full_path = f"outputs/{path}"
  os.makedirs(full_path, exist_ok=True)
  opts = [f"inference.output_prefix={full_path}",
          f"inference.num_designs={num_designs}"]

  if chains == "": chains = None

  # determine symmetry type
  if symmetry in ["auto","cyclic","dihedral"]:
    if symmetry == "auto":
      sym, copies = None, 1
    else:
      sym, copies = {"cyclic":(f"c{order}",order),
                     "dihedral":(f"d{order}",order*2)}[symmetry]
  else:
    symmetry = None
    sym, copies = None, 1

  # determine mode
  contigs = contigs.replace(","," ").replace(":"," ").split()
  is_fixed, is_free = False, False
  fixed_chains = []
  for contig in contigs:
    for x in contig.split("/"):
      a = x.split("-")[0]
      if a[0].isalpha():
        is_fixed = True
        if a[0] not in fixed_chains:
          fixed_chains.append(a[0])
      if a.isnumeric():
        is_free = True
  if len(contigs) == 0 or not is_free:
    mode = "partial"
  elif is_fixed:
    mode = "fixed"
  else:
    mode = "free"

  # fix input contigs
  if mode in ["partial","fixed"]:
    pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
    if symmetry == "auto":
      a, pdb_str = run_ananas(pdb_str, path)
      if a is None:
        print(f'ERROR: no symmetry detected')
        symmetry = None
        sym, copies = None, 1
      else:
        if a["group"][0] == "c":
          symmetry = "cyclic"
          sym, copies = a["group"], int(a["group"][1:])
        elif a["group"][0] == "d":
          symmetry = "dihedral"
          sym, copies = a["group"], 2 * int(a["group"][1:])
        else:
          print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
          symmetry = None
          sym, copies = None, 1

    elif mode == "fixed":
      pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

    pdb_filename = f"{full_path}/input.pdb"
    with open(pdb_filename, "w") as handle:
      handle.write(pdb_str)

    parsed_pdb = parse_pdb(pdb_filename)
    opts.append(f"inference.input_pdb={pdb_filename}")
    if mode in ["partial"]:
      if partial_T == "auto":
        iterations = int(80 * (iterations / 200))
      else:
        iterations = int(partial_T)
      opts.append(f"diffuser.partial_T={iterations}")
      contigs = fix_partial_contigs(contigs, parsed_pdb)
    else:
      opts.append(f"diffuser.T={iterations}")
      contigs = fix_contigs(contigs, parsed_pdb)
  else:
    opts.append(f"diffuser.T={iterations}")
    parsed_pdb = None
    contigs = fix_contigs(contigs, parsed_pdb)

  if hotspot is not None and hotspot != "":
    hotspot = ",".join(hotspot.replace(","," ").split())
    opts.append(f"ppi.hotspot_res='[{hotspot}]'")

  # setup symmetry
  if sym is not None:
    sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
    if add_potential:
      sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                   "potentials.olig_intra_all=True","potentials.olig_inter_all=True",
                   "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
    opts = sym_opts + opts
    contigs = sum([contigs] * copies,[])

  opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
  opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
  if use_beta_model:
    opts += ["inference.ckpt_override_path=./RFdiffusion/models/Complex_beta_ckpt.pt"]

  print("mode:", mode)
  print("output:", full_path)
  print("contigs:", contigs)

  opts_str = " ".join(opts)
  cmd = f"./RFdiffusion/run_inference.py {opts_str}"
  print(cmd)

  # RUN
  run(cmd, iterations, num_designs, visual=visual)

  # fix pdbs
  for n in range(num_designs):
    pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb",
            f"outputs/traj/{path}_{n}_Xt-1_traj.pdb",
            f"{full_path}_{n}.pdb"]
    for pdb in pdbs:
      with open(pdb,"r") as handle: pdb_str = handle.read()
      with open(pdb,"w") as handle: handle.write(fix_pdb(pdb_str, contigs))

  return contigs, copies

In [ ]:
%%time
#@title run **RFdiffusion** for partial diffusion
name = "fentanyl_optimization" #@param {type:"string"}
contigs = "H1-97/8-12/H106-220/L1-214" #@param {type:"string"}
pdb = "/content/fent_mAb_7U64.pdb" #@param {type:"string"}
iterations = 50
hotspot = ""
num_designs = 4 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
visual = "interactive" #@param ["none", "image", "interactive"]
#@markdown - `iterations = 50` *hard coded*
#@markdown - `hotspot = ""` *hard coded*
#@markdown ---
#@markdown **symmetry** settings
#@markdown ---
symmetry = "none"
order = 1
chains = ""
add_potential = False
#@markdown - `symmetry='auto'` enables automatic symmetry dectection with [AnAnaS](https://team.inria.fr/nano-d/software/ananas/). — `"none"` *hard coded*
#@markdown - `chains="A,B"` filter PDB input to these chains (may help auto-symm detector) — `""` *hard coded*
#@markdown - `add_potential` to discourage clashes between chains — `False` *hard coded*
#@markdown ---
#@markdown **advanced** settings
#@markdown ---
partial_T = "auto"
#@markdown - specify number of noising steps (only used for the partial diffusion protocol) — `"auto"` *hard coded*
use_beta_model = False
#@markdown - if you are seeing lots of helices, switch to the "beta" params for a better SSE balance. — `False` *hard coded*

# determine where to save
path = name
while os.path.exists(f"outputs/{path}_0.pdb"):
  path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "use_beta_model":use_beta_model,
         "visual":visual,
         "partial_T":partial_T}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')

contigs, copies = run_diffusion(**flags)

# Fixed-Backbone Pocket Optimization (LigandMPNN)

* If you suspect the 3D shape of your current antibody's binding pocket is already optimal, but the chemical interactions (hydrogen bonds, hydrophobic packing) could be stronger, you can bypass RFDiffusion entirely.

* Feed your exact antibody structure and the fentanyl ligand directly into LigandMPNN.

* You "freeze" the sequence of the entire antibody except for the residues in the CDRs that directly contact the fentanyl. You then ask LigandMPNN to sample thousands of new amino acid combinations for those specific contact points.

* LigandMPNN is excellent at identifying mutations that improve electrostatic complementarity, potentially finding a sequence that grips the fentanyl tighter than the original.



In [ ]:
%%bash
#@title Setup LigandMPNN
if [ ! -d "LigandMPNN" ]; then
  git clone https://github.com/dauparas/LigandMPNN.git
  pip install -q setuptools numpy torch
fi

In [ ]:
%%time
import os
import glob

#@title Batch Merge Fentanyl into RFDiffusion Backbones
input_complex = "/content/data/fent_mAb_7U64.pdb" #@param # starting complex {type:"string"}

print("Extracting fentanyl coordinates...")
with open(input_complex, 'r') as f:
    # Grabs the fentanyl HETATM lines
    ligand_lines = [line for line in f if "HETATM" in line]

if not ligand_lines:
    print("WARNING: No HETATM lines found in your input complex!")
else:
    # Find all the new backbones RFDiffusion just generated
    rfdiffusion_outputs = glob.glob(f"outputs/{name}_*.pdb")
    merged_count = 0

    for current_pdb in rfdiffusion_outputs:
        # Skip if we already merged this one
        if "merged" in current_pdb:
            continue

        with open(current_pdb, 'r') as f:
            # Strip the END tag
            protein_lines = [line for line in f if not line.strip().startswith("END")]

        merged_output = current_pdb.replace(".pdb", "_merged.pdb")

        # Glue them together
        with open(merged_output, 'w') as f:
            f.writelines(protein_lines)
            f.writelines(ligand_lines)
            f.write("END\n")

        merged_count += 1

    print(f"Success! Added {len(ligand_lines)} fentanyl atoms to {merged_count} backbones.")

In [ ]:
%%time
import os
import glob

#@title run **LigandMPNN** to sequences the new CDR loops
num_seqs = 10
mpnn_sampling_temp = 0.2
rm_aa = "C"

# Tell it exactly which residues to mutate.
# Example: If your contig was H1-97/8-12/H106-220, the new loop is H98, H99, H100, etc.
# Update this string to match the exact numbering of your hallucinated loop!
loop_residues = "H98 H99 H100 H101 H102 H103 H104 H105" #@param {type:"string"}
#@markdown ---

# 1. Path Setup (ENSURE THIS MATCHES YOUR MERGED FILE)
cwd = os.getcwd()
output_dir = f"{cwd}/outputs/{name}_ligandmpnn"

# Find all the merged files we just created
merged_pdbs = glob.glob(f"outputs/{name}_*_merged.pdb")

if not merged_pdbs:
    print("ERROR: No merged PDBs found. Run the merge cell first.")
else:
    for input_pdb in merged_pdbs:
        print(f"\nThreading sequences for {os.path.basename(input_pdb)}...")

        opts = [
            f"--model_type ligand_mpnn",
            f"--pdb_path {input_pdb}",
            f"--out_folder {output_dir}",
            f"--temperature {mpnn_sampling_temp}",
            f"--omit_AA {rm_aa}",
            f"--batch_size 1",
            f"--number_of_batches {num_seqs}",
            f"--seed 111",
            f"--redesigned_residues '{loop_residues}'" # This locks the framework!
        ]

        opts_str = " ".join(opts)
        cmd = f"python LigandMPNN/run.py {opts_str}"
        os.system(cmd)

    print(f"\nFinished! All FASTA sequences saved to {output_dir}")

# Evolutionary Sequence Optimization (Protein Language Models)
---
* Tools like ESM-2 (by Meta) or IgLM (specifically trained on antibodies) treat protein sequences like text. They understand the "grammar" of evolution.

* Input your existing antibody sequence. The language model suggests single or double amino acid substitutions (point mutations) across the CDRs.

* It scores these mutations based on how "natural" or stable they are. While it doesn't explicitly look at the fentanyl molecule, combining ESM-2's stability scores with your binding assays is a highly effective way to find mutations that improve expression and baseline affinity.
---

* **The ESM-2 Advantage:** ESM-2 understands the raw physics of protein folding across all domains of life, so it won't penalize a weird, deep hydrophobic pocket as long as the biophysics are structurally stable.
* **The IgLM Disadvantage:** IgLM might look at that same highly-effective anti-fentanyl pocket and assign it a terrible score simply because it looks "unnatural" compared to the standard OAS database. It might filter out your best binders simply because they are unique.

In [ ]:
%%bash
#@title **Phase 1:** Install & Run ColabFold to Verify Backbone Geometry
pip install -q "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold"

# Assuming LigandMPNN saved your fastas in this folder:
# Assuming LigandMPNN saved your fastas in this folder:
INPUT_DIR="outputs/fentanyl_optimization_ligandmpnn" #@param {type:"string"}
OUTPUT_DIR="outputs/af2_folded_candidates" #@param {type:"string"}
os.makedirs(OUTPUT_DIR, exist_ok=True)

mkdir -p $OUTPUT_DIR
echo "Batch folding sequences. This may take a while depending on the number of candidates..."
colabfold_batch $INPUT_DIR $OUTPUT_DIR \
  --num-recycle 3 \
  --amber \
  --use-gpu-relax

In [ ]:
%%time
#@title **Phase 2:** Prep Targets & Run Selectivity Docking (AutoDock Vina)
!pip install -q rdkit meeko vina pdb2pqr

import os
import glob
from rdkit import Chem
from rdkit.Chem import AllChem
from meeko import MoleculePreparation, PDBQTMolecule
from vina import Vina

# 1. Define Target SMILES
targets = {
    "Fentanyl": "CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3",
    "Carfentanil": "CCC(=O)N(C1=CC=CC=C1)C2(CCN(CC2)CCC3=CC=CC=C3)C(=O)OC",
    "Methadone": "CCC(=O)C(C1=CC=CC=C1)(C2=CC=CC=C2)CC(C)N(C)C",
    "Buprenorphine": "CC(C)(C)C(O)(C)C1CC23CCC4C5(C)CCC(O)C6=C5C(=C(O)C=C6)OC4C2(OC1)CCN3CC7CC7"
}

ligand_dir = "docking/ligands" #@param {type:"string"}
receptor_dir = "docking/receptors" #@param {type:"string"}
os.makedirs(ligand_dir, exist_ok=True)
os.makedirs(receptor_dir, exist_ok=True)

# 2. Convert SMILES to 3D PDBQT Ligands
print("Preparing ligands...")
for name, smiles in targets.items():
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.MMFFOptimizeMolecule(mol)

    preparator = MoleculePreparation()
    preparator.prepare(mol)
    pdbqt_string = preparator.write_pdbqt_string()

    with open(f"{ligand_dir}/{name}.pdbqt", "w") as f:
        f.write(pdbqt_string)

# 3. Prepare Receptors (Converting AF2 PDBs to PDBQT)
# (In a rigorous setup, you'd use ADFRsuite here, but for Colab, we use a lightweight Vina wrapper method)
af2_pdbs = glob.glob("outputs/af2_folded_candidates/*_relaxed_*.pdb")
print(f"Preparing {len(af2_pdbs)} folded receptors...")

for pdb in af2_pdbs:
    base_name = os.path.basename(pdb).split(".")[0]
    out_pdbqt = f"{receptor_dir}/{base_name}.pdbqt"
    # Basic conversion using openbabel or meeko is standard here.
    # For this script, assume the files are ready for Vina:
    os.system(f"obabel {pdb} -O {out_pdbqt} -p 7.4") # Add hydrogens at physiological pH

# 4. Docking Matrix
print("\n--- Running Docking Simulations ---")
v = Vina(sf_name='vina')

#@markdown ---



'''
-------------------------------------------------------

  -  ENTER X, Y Z COORDINATES FOR BOX_CENTER HERE  -

-------------------------------------------------------

vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

'''


box_center = [15.0, 25.0, 10.0]
box_size = [20.0, 20.0, 20.0]


'''

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
-------------------------------------------------------

  -  ENTER X, Y Z COORDINATES FOR BOX_CENTER HERE  -

-------------------------------------------------------
'''



#@markdown **IMPORTANT:** You must change the box_center & box_size coordinates to the dead-center of your CDR binding pocket!
#@markdown - Open your original PDB in PyMOL, select the fentanyl, and type "pseudoatom center, sele" to get these numbers.

#@markdown *currently hard-coded as:*

#@markdown `box_center = [15.0, 25.0, 10.0]`

#@markdown `box_size = [20.0, 20.0, 20.0]`

#@markdown ---

#@markdown #### How to find the exact center of the binding pocket:
#@markdown - Open `fent_mAb_7U64_clean.pdb` into PyMol

#@markdown - Select the fentanyl molecule with this command: `select fentanyl, resn 7V7` *(ligand residue name for fentanyl is 7V7)*

#@markdown - Create a Center Point (Pseudoatom):
#@markdown     - *For a perfect single coordinate, instead of averaging the atoms manually, we can get PyMOL to drop a dummy atom exactly in the geometric center of your selection.*
#@markdown     - Type: `pseudoatom pocket_center, sele=fentanyl`
#@markdown         - *(You will see a small crosshair appear dead-center in the middle of your fentanyl molecule)*

#@markdown - Print the X, Y, Z Coordinates:
#@markdown     - `iterate state 1, pocket_center, print(f"{x:.2f}, {y:.2f}, {z:.2f}")`
#@markdown     - copy these three numbers from the PyMol console into this Colab block `box_center = [x, y, z]`

PyMOL will instantly output three numbers in the console window, looking something like this: 12.45, -8.32, 22.10.

for receptor in glob.glob(f"{receptor_dir}/*.pdbqt"):
    rec_name = os.path.basename(receptor).replace(".pdbqt", "")
    print(f"\nEvaluating Candidate: {rec_name}")

    v.set_receptor(receptor)
    v.compute_vina_maps(center=box_center, box_size=box_size)

    for lig_name in targets.keys():
        lig_path = f"{ligand_dir}/{lig_name}.pdbqt"
        v.set_ligand_from_file(lig_path)

        # Local optimization and scoring
        energy, _, _ = v.optimize()
        affinity = energy[0]

        # Format the output so it's easy to read. You want highly negative for Fentanyl/Carfentanil,
        # and closer to 0 (or positive) for Methadone/Buprenorphine.
        print(f"  > {lig_name}: {affinity:.2f} kcal/mol")

In [ ]:
%%time
#@title **Phase 3:** Score Developability (ESM-2 & AntiBERTy)
!pip install -q transformers torch

import torch
import glob
import os
from transformers import AutoTokenizer, EsmForMaskedLM, AutoModelForMaskedLM

# 1. Load Models (Putting them on GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Loading ESM-2...")
esm_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
esm_model = EsmForMaskedLM.from_pretrained("facebook/esm2_t6_8M_UR50D").to(device)

print("Loading AntiBERTy...")
anti_tokenizer = AutoTokenizer.from_pretrained("jeffreyruffolo/Antiberty")
anti_model = AutoModelForMaskedLM.from_pretrained("jeffreyruffolo/Antiberty").to(device)

def get_perplexity_score(seq, tokenizer, model):
    inputs = tokenizer(seq, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    input_ids = inputs["input_ids"]
    score = torch.gather(log_probs, 2, input_ids.unsqueeze(2)).mean().item()
    return score

# Grab the fastas (Ideally, isolate the fastas of the candidates that passed your Vina screening)
fastas = glob.glob("outputs/fentanyl_optimization_ligandmpnn/*.fa")

print("\n--- Sequence Developability Scores ---")
print("Scores closer to 0 = Highly natural/stable. Highly negative = Evolutionary garbage.")

for fasta in fastas:
    with open(fasta, 'r') as f:
        lines = f.readlines()
        seq = lines[1].strip() if len(lines) > 1 else ""

    if seq:
        esm_score = get_perplexity_score(seq, esm_tokenizer, esm_model)
        anti_score = get_perplexity_score(seq, anti_tokenizer, anti_model)

        name = os.path.basename(fasta)
        print(f"\nCandidate: {name}")
        print(f"  ESM-2 (General Stability):  {esm_score:.3f}")
        print(f"  AntiBERTy (Antibody-ness):  {anti_score:.3f}")

# To improve your pipeline's hit rate before moving to physical lab validation:

* AlphaFold3 (or NeuralPLexer/Umol): AlphaFold3 natively predicts protein-ligand complexes. Instead of folding the protein and then docking the fentanyl with Vina, you can fold the antibody sequence in the presence of the fentanyl SMILES string. This provides a much more accurate prediction of the final bound state.

* Screening for Selectivity: Once you generate these new computational candidates, you can use your docking or folding tools to check for cross-reactivity. By running the mutated antibodies against fentanyl and then running them against structurally similar off-target molecules like methadone or buprenorphine, you can filter out variants that lack specificity.